**# Step 1: Import Required Libraries:**

In [1]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import requests
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import tensorflow as tf

**# Step 2: Define Paths and Load Metadata**

In [7]:
DATA_DIR = r'C:\Users\iminy\OneDrive\Desktop\AIML\SKIN'
IMAGE_DIR = os.path.join(DATA_DIR, 'images')
CSV_PATH = os.path.join(DATA_DIR, 'GroundTruth.csv')

# Load metadata
df = pd.read_csv(CSV_PATH)

# Map image to image path
image_files = os.listdir(IMAGE_DIR)
image_dict = {f.split('.')[0]: os.path.join(IMAGE_DIR, f) for f in image_files}
df['image_path'] = df['image'].map(image_dict)

# Handle missing images
df = df.dropna(subset=['image_path'])

**# Step 3: Map Diagnosis Columns to Numerical Labels**

In [8]:
diagnosis_columns = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
df['label'] = df[diagnosis_columns].idxmax(axis=1)
lesion_type = {
    'NV': 0,
    'MEL': 1,
    'BKL': 2,
    'BCC': 3,
    'AKIEC': 4,
    'DF': 5,
    'VASC': 6
}
df['label'] = df['label'].map(lesion_type)

**# Step 4: Split Data into Training, Validation, and Test Sets**

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42, stratify=train_df['label'])

IMG_SIZE = (224, 224)

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0
    return img

# Load and preprocess images
train_images = np.array([load_and_preprocess_image(path) for path in train_df['image_path']])
val_images = np.array([load_and_preprocess_image(path) for path in val_df['image_path']])
test_images = np.array([load_and_preprocess_image(path) for path in test_df['image_path']])

train_labels = np.array(train_df['label'])
val_labels = np.array(val_df['label'])
test_labels = np.array(test_df['label'])


**# Step 5: Data Augmentation**

In [10]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(train_images)

**# Step 6: Load Pre-trained Model**

In [28]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch

try:
    processor = AutoImageProcessor.from_pretrained("NeuronZero/SkinCancerClassifier")
    model = AutoModelForImageClassification.from_pretrained(
        "NeuronZero/SkinCancerClassifier",
        num_labels=7,
        id2label={0: "NV", 1: "MEL", 2: "BKL", 3: "BCC", 4: "AKIEC", 5: "DF", 6: "VASC"},
        label2id={"NV": 0, "MEL": 1, "BKL": 2, "BCC": 3, "AKIEC": 4, "DF": 5, "VASC": 6}
    )

    # Ensure that the model is in evaluation mode and moved to the correct device (CPU/GPU)
    model.eval()
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    print("Pre-trained model loaded successfully.")
except Exception as e:
    print(f"Error loading pre-trained model: {e}")


Error loading pre-trained model: 
AutoModelForImageClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForImageClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.

